In [26]:
import pandas as pd

In [77]:
df = pd.read_csv('../data/raw/scraped_cian_raw.csv')

In [78]:
df.nunique()

author                 1316
author_type               7
url                    7047
location                  1
deal_type                 1
accommodation_type        1
floor                    53
floors_count             67
rooms_count               5
total_meters           1835
price_per_month           1
commissions               1
price                  4776
district                137
street                 1037
house_number           1094
underground             293
residential_complex     556
dtype: int64

# Убираем ненужные фичи

In [79]:
df = df.drop_duplicates().drop(columns=['location', 'deal_type', 'accommodation_type', 'url', 'price_per_month', 'commissions', 'author', 'house_number', 'residential_complex'])

In [80]:
df

,author_type,floor,floors_count,rooms_count,total_meters,price,district,street,underground
0,developer,-1,-1,-1,1.00,25544778.0,Щукино,Щукинская,Щукинская
1,real_estate_agent,10,14,1,65.20,35000000.0,Мещанский,Гиляровского,Проспект Мира
2,developer,18,41,1,37.70,24335267.0,Даниловский,Дубининская,Павелецкая
3,real_estate_agent,23,23,1,54.00,29000000.0,Хорошево-Мневники,Нижние Мневники,Терехово
4,developer,2,14,1,30.45,29201550.0,Дорогомилово,1-й Можайский тупик,Киевская
...,...,...,...,...,...,...,...,...,...
7044,real_estate_agent,6,12,1,25.10,6500000.0,Печатники,Гурьянова,Депо
7045,real_estate_agent,1,5,1,19.40,6200000.0,Измайлово,Измайловский проезд,Измайловская
7046,real_estate_agent,2,5,1,19.80,6400000.0,Тимирязевский,Вучетича,Гражданская
7047,real_estate_agent,13,28,-1,24.08,5690000.0,Митино,Пятницкое шоссе,Пятницкое шоссе


# Далее убираем все примеры, содержащие NaN или "-1"

In [81]:
indexes = df.loc[df['floors_count'] == -1].index
df = df.drop(indexes)

In [82]:
indexes = df.loc[df['total_meters'] <= 9.00].index
df = df.drop(indexes)

In [83]:
df = df.dropna(subset="price")

In [84]:
indexes = df.loc[df['rooms_count'] == -1].index
df = df.drop(indexes)

In [85]:
indexes = df.loc[df['floor'] == -1].index
df = df.drop(indexes)

Нужно дропнуть строки в которых и 'underground' NaN и 'district' NaN, потом посмотреть где еще остаются какие пробелы

In [86]:
indexes = df[(df['underground'].isna()) & (df['district'].isna())].index
df = df.drop(indexes)

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6554 entries, 1 to 7048
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   author_type   6296 non-null   object 
 1   floor         6554 non-null   int64  
 2   floors_count  6554 non-null   int64  
 3   rooms_count   6554 non-null   int64  
 4   total_meters  6554 non-null   float64
 5   price         6554 non-null   float64
 6   district      4214 non-null   object 
 7   street        4027 non-null   object 
 8   underground   6432 non-null   object 
dtypes: float64(2), int64(3), object(4)
memory usage: 512.0+ KB


In [88]:
df = df[df.notna().sum(axis=1) == 9]

In [64]:
import requests

def fetch_coordinates(apikey, address):
    base_url = "https://geocode-maps.yandex.ru/1.x"
    response = requests.get(base_url, params={
        "geocode": address,
        "apikey": apikey,
        "format": "json",
    })
    response.raise_for_status()
    found_places = response.json()['response']['GeoObjectCollection']['featureMember']

    if not found_places:
        return None

    most_relevant = found_places[0]
    lon, lat = most_relevant['GeoObject']['Point']['pos'].split(" ")
    return lon, lat

In [72]:
apikey = ''  # ваш ключ

coords = fetch_coordinates(apikey, "Проспект Мира")
print(coords)  # ('37.295014', '55.608562')

coords = fetch_coordinates(apikey, "Серпуховская")
print(coords)  # ('37.624992', '55.726872')

coords = fetch_coordinates(apikey, "Красная площадь")
print(coords)  # ('37.621031', '55.753595')

coords = fetch_coordinates(apikey, "UNEXIST")
print(coords)  # None

('37.637964', '55.812529')
('37.6251', '55.727293')
('37.621202', '55.753544')
None


In [67]:
coords = list()
for underground in df['underground']:
    coords.append(fetch_coordinates(apikey, underground))


In [68]:
df['coordinates_of_underground'] = coords

In [69]:
df

,author_type,floor,floors_count,rooms_count,total_meters,price,district,street,underground,coordinates_of_underground
1,real_estate_agent,10,14,1,65.20,35000000.0,Мещанский,Гиляровского,Проспект Мира,"(37.637964, 55.812529)"
2,developer,18,41,1,37.70,24335267.0,Даниловский,Дубининская,Павелецкая,"(37.64736, 55.713228)"
3,real_estate_agent,23,23,1,54.00,29000000.0,Хорошево-Мневники,Нижние Мневники,Терехово,"(38.055429, 51.342904)"
4,developer,2,14,1,30.45,29201550.0,Дорогомилово,1-й Можайский тупик,Киевская,"(30.476622, 50.023928)"
5,real_estate_agent,4,14,1,45.00,25750000.0,Раменки,проспект Генерала Дорохова,Минская,"(27.461166, 53.387641)"
...,...,...,...,...,...,...,...,...,...,...
7043,real_estate_agent,2,5,1,17.00,7300000.0,Красносельский,Московско-Казанский переулок,Красносельская,"(63.676109, 53.2324)"
7044,real_estate_agent,6,12,1,25.10,6500000.0,Печатники,Гурьянова,Депо,"(36.842084, 60.915985)"
7045,real_estate_agent,1,5,1,19.40,6200000.0,Измайлово,Измайловский проезд,Измайловская,"(37.781506, 55.787761)"
7046,real_estate_agent,2,5,1,19.80,6400000.0,Тимирязевский,Вучетича,Гражданская,"(47.20177, 56.122916)"


In [70]:
df.nunique()

author_type                      7
floor                           52
floors_count                    61
rooms_count                      4
total_meters                  1102
price                         1713
district                       124
street                         889
underground                    270
coordinates_of_underground     269
dtype: int64

In [89]:
df.to_csv("../data/interim/scraped_cian_interim.csv", index = False)